In [1]:
# It is a framework that allows you to build applications using LLM

In [3]:
from secret_key import openai
import os 

os.environ['OPENAI_API_KEY']= openai

In [3]:
# ! pip install langchain

In [4]:
# ! pip install openai

In [4]:
from langchain.llms import OpenAI

# it tells that how creative you want your model to be, its bascially the risk parameter. More the risk their is a chance 
# that we will get wrong output, but the answer will be more creative.
llm = OpenAI(temperature= 0.6)

name= llm('I want to open a Italian restaurant, suggest me a single fancy name')
print(name)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Simple LLMChain

In [6]:
# to avoid changing the cuisine name , lets have a template 
from langchain.prompts import PromptTemplate

prompt_temp_name= PromptTemplate(
    input_variables= ['cuisine'],
    template= 'I want to open a {cuisine} restaurant, suggest me a fancy name'
)

prompt_temp_name.format(cuisine='Mexican')

'I want to open a Mexican restaurant, suggest me a fancy name'

In [7]:
# Make a connection of the Llm and the prompt template

from langchain.chains import LLMChain

chain= LLMChain(llm = llm, prompt= prompt_temp_name)
chain.run('American')

C:\Users\varsha\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"Stateside Eats" '

### Simple Sequential Chain: Gives only one output (example only the food item menu)

##### Simple chain where the outputs of one step feed directly into next.


In [8]:
# to avoid changing the cuisine name , lets have a template 
from langchain.prompts import PromptTemplate

prompt_temp_name= PromptTemplate(
    input_variables= ['cuisine'],
    template= "I want to open a {cuisine} restaurant, suggest me a fancy name"
)

name_chain= LLMChain(llm = llm, prompt= prompt_temp_name)

# prompt for generating menu
food_menu= PromptTemplate(
            input_variables= ['restaurantName'],
            template= "Suggest some menu items for {restaurantName}. Return it as a comma seprated list."
)

food_item_chain= LLMChain(llm = llm, prompt= food_menu)

In [9]:
# lets create a chain to connect both the sentences in one 

from langchain.chains import SimpleSequentialChain

chain= SimpleSequentialChain(chains= [name_chain, food_item_chain] )
response= chain.run('Indian') 
print(response)



1. Maharaja's Palace: Chicken Tikka Masala, Lamb Biryani, Palak Paneer, Garlic Naan, Mango Lassi
2. Spice Kingdom: Vegetable Samosas, Butter Chicken, Lamb Vindaloo, Vegetable Korma, Mango Chutney
3. Tandoori Nights: Tandoori Chicken, Chicken Tikka, Lamb Seekh Kebab, Vegetable Biryani, Garlic Naan
4. Curry Garden: Sambar, Chicken Curry, Aloo Gobi, Vegetable Pakoras, Mango Lassi
5. Namaste India: Chicken Tikka Masala, Lamb Rogan Josh, Saag Paneer, Vegetable Biryani, Garlic Naan
6. Royal Masala: Chicken Tikka, Lamb Korma, Chana Masala, Vegetable Pulao, Mango Lassi
7. Bollywood Bites: Butter Chicken, Lamb Biryani, Aloo Tikki, Vegetable Jalfrezi, Mango Lassi
8. Garam Masala: Chicken Vindaloo, Lamb Saag, Vegetable Biryani, Onion Bhajis, Mango Lassi
9. The Curry Club: Chicken Korma, Lamb Madras, Vegetable Jalfrezi,


## Sequential Chain: Multiple input and multiple output (restraurant name and food menu)

##### Chain where the outputs of one chain feed directly into next.

In [10]:
# to avoid changing the cuisine name , lets have a template 
from langchain.prompts import PromptTemplate

llm= OpenAI(temperature= 0.7)
prompt_temp_name= PromptTemplate(
    input_variables= ['cuisine'],
    template= "I want to open a {cuisine} restaurant, suggest me a fancy name"
)

name_chain= LLMChain(llm = llm, prompt= prompt_temp_name, output_key= "restaurant_name")

# prompt for generating menu
llm= OpenAI(temperature= 0.7)

prompt_food_menu= PromptTemplate(
            input_variables= ['restaurant_name'],
            template= "Suggest some menu items for {restaurant_name}. Return it as a comma seprated list."
)

food_item_chain= LLMChain(llm= llm, prompt= prompt_food_menu, output_key= 'food_items')

In [11]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains= [name_chain, food_item_chain],
    input_variables= ['cuisine'],
    output_variables= ['restaurant_name', 'food_items']
    
)

chain({'Indian'})

C:\Users\varsha\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'cuisine': {'Indian'},
 'restaurant_name': '\n\n"Spice Palace" ',
 'food_items': '\n\n1. Tandoori Chicken\n2. Butter Chicken\n3. Lamb Vindaloo\n4. Vegetable Samosas\n5. Chicken Tikka Masala\n6. Aloo Gobi\n7. Garlic Naan\n8. Palak Paneer\n9. Chana Masala\n10. Mango Lassi'}

## LangChain Agents Using Wikipedia

##### Tools allow agents to interact with various resources and services like APIs, databases, file systems, etc.

In [12]:
# ! pip install wikipedia

In [13]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm= OpenAI(temperature= 0.7)

# load the tools from the llm
tools= load_tools(['wikipedia', 'llm-math'], llm=llm)

agent= initialize_agent(
        tools,
        llm,
        agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION,    # this means, as we humans has always have a thought before a
        verbose= True                                    # reaction, similarly agents try to mimc it,
                                       # eg, first we wil lhave the thought of where to go and than we will react on it
)

# verbose= True will give you the steps followed by the llm model to find the output.... we can use it for all the 
# above models too!

agent.run("When was Elon Musk war born?and what is his age in 2023?")

C:\Users\varsha\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use Wikipedia to look up Elon Musk and his birthdate
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to look up Elon Musk's birthdate
Action: Wikipedia
Action Input: Elon Musk's birthdate
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to look up Elon Musk's age in 2023
Action: Wikipedia
Action Input: Elon Musk's age in 2023
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use the Calculator to calculate Elon Musk's age in 2023
Action: Calculator
Action Input: 2023 - 1971
Observation: Answer: 52
Thought: I now know the final answer
Final Answer: Elon Musk was born on June 28, 1971 and his age in 2023 will be 52 years old.

> Finished chain.


'Elon Musk was born on June 28, 1971 and his age in 2023 will be 52 years old.'

## LangChain Agent modeling using SerpApi

#### SerpApi is a google Search Api, ie. whatever we do serach on google and whatever search output we get from it, we can use it programitcally.

In [14]:
# ! pip install google-search-results

In [15]:
from serpapi import serpapi_key

os.environ['SERPAPI_API_KEY']= serpapi_key

In [16]:
llm = OpenAI(temperature= 0.7)

tools= load_tools(['serpapi','llm-math'], llm= llm)

agent= initialize_agent(
    tools,
    llm, 
    agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose= True
)

agent.run("What was the Inid GDP in 2023 +5?")

ValidationError: 1 validation error for SerpAPIWrapper
__root__
  Could not import serpapi python package. Please install it with `pip install google-search-results`. (type=value_error)

## LLM Memory

In [ ]:
dir(chain)

In [ ]:
type(chain.memory)

In [ ]:
# we need to attach the memory we need to create an additional memory object and attach it to thiis

In [ ]:
from langchain.memory import ConversationBufferMemory

memory= ConversationBufferMemory()

temp= PromptTemplate(
    input_variables=['actor'],
    template= 'Give me one name of a movie, in this {actor} is one of the lead hero'
)

chain= LLMChain(llm= llm, prompt= temp, memory= memory)
chain.run('Tony Stark')

In [ ]:
chain= LLMChain(llm= llm, prompt= temp, memory= memory)
chain.run('Shahid Kapoor')

In [ ]:
chain.run('Prabhas')

In [ ]:
print(chain.memory.buffer)

In [ ]:
# one problem with this conversational buffer memory is that it will keep growing exponentially

# one pair of que and ans is called as one conversational exchange, so when next time we go and ask question to 
# open- ai, than it is goint to send all this conversational history to open-ai, and open-ai charges  per token, 
# which will increase the cost.

## Langchain Conversation Chain

#### this will restrict the buffer the size, that is the overcome for the above issue

In [ ]:
from langchain.chains import ConversationChain

convo= ConversationChain(llm = OpenAI(temperature= 0.7))

In [ ]:
# default prompt that is present in it
convo.prompt

In [ ]:
print(convo.prompt.template)

In [ ]:
convo.run("what were thw lead characters in Avengers End Game?")

In [ ]:
convo.run("what is 9+10 ?")

In [ ]:
convo.run('who used hammer as weapon in that movie?')

In [ ]:
# here we dont have to create a memory buffer seperately as it allready has a buffer attached to it, you can see below

convo.memory

In [ ]:
print(convo.memory.buffer)

In [ ]:
# again it will create a endless convo, so to restrict it the buffer size

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory= ConversationBufferWindowMemory(k=1) # we are telling to remember only the last one convo window

convo= ConversationChain(
    llm= OpenAI(temperature= 0.7),
    memory= memory
)

convo.run("what were the lead characters in Avengers End Game?")

In [ ]:
convo.run(("what is 5+5?"))

In [ ]:
print(convo.memory.buffer)

In [ ]:
# now, if we try to ask someother ques regarding the Avenger movie, it wont have the answer

convo.run('who used was the lead actor in that movie?')